## Working with websites that have js interaction

In [1]:
import requests
url="https://www.naukri.com/data-science-jobs"
resp = requests.get(url).text
print(resp)

<!DOCTYPE html> <html lang="en"> <head> <meta http-equiv="X-UA-Compatible" content="IE=Edge" /> <meta name="viewport" content="width=device-width,initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" /> <link rel="dns-prefetch preconnect" href="//static.naukri.com"> <link rel="dns-prefetch preconnect" href="//img.naukri.com"> <link rel="dns-prefetch preconnect" href="//logs.naukri.com"> <link rel="dns-prefetch preconnect" href="//lg.naukri.com"> <link rel="dns-prefetch" href="https://www.googletagmanager.com" /> <base href="//www.naukri.com" /> <link rel="manifest" href="./manifest.json"> <link rel="preload" as="style" href="//static.naukri.com/s/7/109/c/main.f59532c6.min.css"/> <link rel="preload" as="script" href="//static.naukri.com/s/7/109/j/main.3d561068.min.js"> <script>
        window.ipAddress = "www.naukri.com";
        window.DEBUG = true;
        window.appId = 135;
                    var isCrawler = false;
                            var ucBrowser = fa

In [2]:
import os
from selenium import webdriver
#os.chdir("./chrome_driver") ### WINDOWS PEOPLE
browser = webdriver.Chrome()

In [3]:
browser.get(url)

In [5]:
raw_html = browser.page_source

In [6]:
from bs4 import BeautifulSoup
parsed_html = BeautifulSoup(raw_html,'html.parser')

In [12]:
## Extract the Job Title: Spend time on figuring out the correct code to extract job title.
## salary
## exp
## company
## loc

In [29]:
titles = []
exps = []
sals = []
locs = []
comps = []
for i in parsed_html.find_all("article",class_="jobTuple bgWhite br4 mb-8"):
    for k in i.find_all("div",class_="jobTupleHeader"):
        titles.append(k.a.text)
    exps.append(i.find_all("li")[0].text)
    sals.append(i.find_all("li")[1].text)
    locs.append(i.find_all("li")[2].text)
    comps.append(i.find_all('a')[1].text)

In [32]:
import pandas as pd
table = pd.DataFrame({'titles':titles,'experience':exps,'salary':sals,'location':locs,'company':comps})

In [33]:
def get_data(parsed_html):
    titles = []
    exps = []
    sals = []
    locs = []
    comps = []
    for i in parsed_html.find_all("article",class_="jobTuple bgWhite br4 mb-8"):
        for k in i.find_all("div",class_="jobTupleHeader"):
            titles.append(k.a.text)
        exps.append(i.find_all("li")[0].text)
        sals.append(i.find_all("li")[1].text)
        locs.append(i.find_all("li")[2].text)
        comps.append(i.find_all('a')[1].text)
    table = pd.DataFrame({'titles':titles,'experience':exps,'salary':sals,'location':locs,'company':comps})
    return table

In [34]:
page_num=1
base_url = "https://www.naukri.com/data-science-jobs-{}".format(page_num)
print(base_url)

https://www.naukri.com/data-science-jobs-1


In [35]:
browser.close()

In [36]:
from tqdm import tqdm
tables = []
for page_num in tqdm(range(1,4)):
    base_url = "https://www.naukri.com/data-science-jobs-{}".format(page_num)
    browser = webdriver.Chrome()
    browser.get(base_url)
    raw_html = browser.page_source
    parsed_html = BeautifulSoup(raw_html,'html.parser')
    tables.append(get_data(parsed_html))
    browser.close()

100%|█████████████████████████████████████████████| 3/3 [01:55<00:00, 38.54s/it]


In [41]:
big_table = pd.concat(tables)

In [60]:
big_table.head(2)

,titles,experience,salary,location,company
0,Data Engineer | Disruptive Digital Healthcare ...,3-6 Yrs,"15,00,000 - 25,00,000 PA.",Bangalore/Bengaluru,Unnati
1,Data Science Sr. Analyst,2-7 Yrs,Not disclosed,Bangalore/Bengaluru,Dell International Services India Private Lim...


In [43]:
### Scrapping html tables
url = "https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250"
browser = webdriver.Chrome()
browser.get(url)

In [44]:
raw_html = browser.page_source
parsed_html = BeautifulSoup(raw_html,'html.parser')

In [48]:
tab = parsed_html.find_all('tbody',class_="lister-list")

In [49]:
len(tab)

1

In [55]:
titles = []
years = []
ratings = []
for row in tab[0].find_all("tr"):
    titles.append(row.find_all("td")[1].a.text)
    years.append(row.find_all("td")[1].span.text)
    ratings.append(row.find_all("td")[2].text.strip())       

In [58]:
pd.DataFrame({'titles':titles,'years':years,'ratings':ratings}).head(2)

,titles,years,ratings
0,Planet Earth II,(2016),9.5
1,Planet Earth,(2006),9.4


In [61]:
### Wikipedia tables
url = "https://en.wikipedia.org/wiki/List_of_terrorist_incidents_in_2019"
browser = webdriver.Chrome()
browser.get(url)

In [62]:
raw_html = browser.page_source
parsed_html = BeautifulSoup(raw_html,'html.parser')

In [63]:
tables = parsed_html.find_all("table",class_="wikitable")

In [64]:
len(tables)

1

In [67]:
rows = tables[0].tbody.find_all("tr")[1:]

In [68]:
dates = []
types = []
deads = []
injured = []
locations = []
articles = []
details = []
perpetrators = []
parts_of = []
cols = [dates,types,deads,injured,locations,articles,details,perpetrators,parts_of]
for row in rows:
    data=row.find_all("td")
    for col,dat in zip(cols,data):
        col.append(dat.text.strip())

In [75]:
table=pd.DataFrame({'date':dates,
             'type':types,
             'dead':deads,
             'injured':injured,
             'article':articles,
             'detail':details,
             'perpetrators':perpetrators,
             'parts_of':parts_of})

In [100]:
def get_data(table):
    rows = table.tbody.find_all("tr")[1:]
    dates = []
    types = []
    deads = []
    injured = []
    locations = []
    articles = []
    details = []
    perpetrators = []
    parts_of = []
    #cols = [dates,types,deads,injured,locations,articles,details,perpetrators,parts_of]
    for row in rows:
        data=row.find_all("td")
        try:
            dates.append(data[0].text.strip())
        except:
            dates.append("No data")
        try:
            types.append(data[1].text.strip())
        except:
            types.append("No data")
        try:
            deads.append(data[2].text.strip())
        except:
            deads.append("No data")
        try:
            injured.append(data[3].text.strip())
        except:
            injured.append("No data")
        try:
            locations.append(data[4].text.strip())
        except:
            locations.append("No data")
        try:
            articles.append(data[5].text.strip())
        except:
            articles.append("No data")
        try:
            details.append(data[6].text.strip())
        except:
            details.append("No data")
        try:
            perpetrators.append(data[7].text.strip())
        except:
            perpetrators.append("No data")
        try:
            parts_of.append(data[8].text.strip())
        except:
            parts_of.append("No data")
            
            
    table=pd.DataFrame({'date':dates,
             'type':types,
             'dead':deads,
             'injured':injured,
             'article':articles,
             'detail':details,
             'perpetrators':perpetrators,
             'parts_of':parts_of})
    return table


In [84]:
headers = []
for i in tables[0].tbody.find_all("tr")[0]:
    headers.append(i.text.strip().strip())

In [86]:
headers = [h for h in headers if h !=""]

In [93]:
pd.DataFrame({k:v for k,v in zip(headers,cols)}).head(2)

,Date,Type,Dead,Injured,Location,Article,Details,Perpetrator,Part of
0,15 January,"Suicide bombing, shootings, hostage taking",21 (+5),28,"Nairobi, Kenya",2019 Nairobi hotel attack,At least one suicide bomber stormed the luxury...,Al-Shabaab,Terrorism in Kenya
1,16 January,Suicide Bombing,19 (+1),3,"Manbij, Syria",2019 Manbij bombing,19 people were killed in a suicide bomb attack...,Islamic State,Syrian Civil War


In [95]:
browser.close()

In [101]:
#https://en.wikipedia.org/wiki/List_of_terrorist_incidents_in_2015
#https://en.wikipedia.org/wiki/List_of_terrorist_incidents_in_2016
#https://en.wikipedia.org/wiki/List_of_terrorist_incidents_in_2017
url="https://en.wikipedia.org/wiki/List_of_terrorist_incidents_in_2016"
browser = webdriver.Chrome()
browser.get(url)
raw_html=browser.page_source
parsed_html = BeautifulSoup(raw_html,'html.parser')
tables = parsed_html.find_all("table",class_="wikitable")
get_data(tables[0])
browser.close()